<a href="https://colab.research.google.com/github/amirpaia/blenderbot/blob/main/blender_90m_with_double_sized_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Installing prerequisites

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
mydrive_path = '/content/drive/MyDrive/colabs/blender-models/'
# !pip uninstall -q parlai
!pip install -q parlai
!pip install transformers

# 1. Preparing Data

In [23]:
def transfer_list_of_turns_to_dialog(d):
    if len(d)%2 !=0: d = d[:-1]
    t = ""
    for i in range(0,len(d),2):
        u1 = d[i]
        u2 = d[i+1]

        if (i+2) != len(d):
            t += "text:"+u1+"\t"+"labels:"+u2+"\n"
        else:
            t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"
    return t

def transfer_list_of_pairs_to_dialog(d):
  t = ""
  for i, text_label_pair in enumerate(d):
    u1 = text_label_pair[0]
    u2 = text_label_pair[1]

    if i != (len(d) - 1):
      t += "text:"+u1+"\t"+"labels:"+u2+"\n"
    else:
      t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"

  return t
    
t = ['hello','how are you','good','bye','test']
print(transfer_list_of_turns_to_dialog(t))

t = [['hello','how are you'],['good','bye']]
print(transfer_list_of_pairs_to_dialog(t))

text:hello	labels:how are you
text:good	labels:bye	episode_done:True

text:hello	labels:how are you
text:good	labels:bye	episode_done:True



## XPersona

In [28]:
!git clone https://github.com/HLTCHKUST/Xpersona.git

import json
import pandas as pd

with open('Xpersona/dataset/Fr_persona_train_corrected.json','r') as f:
   train_data = json.load(f)

dialogs_train = pd.DataFrame(train_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_valid_human_annotated.json','r') as f:
   valid_data = json.load(f)

dialogs_valid = pd.DataFrame(valid_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_test_human_annotated.json','r') as f:
   test_data = json.load(f)

dialogs_test = pd.DataFrame(test_data)['dialogue'].tolist()

# dialogs

rm: cannot remove '/content/dataset_french_xpersona/': No such file or directory
fatal: destination path 'Xpersona' already exists and is not an empty directory.


In [29]:
data_path = "/content/dataset_french_xpersona/"
!rm -R $data_path
!mkdir $data_path

#region Training
data_train = ""
for d in dialogs_train:
  data_train += transfer_list_of_pairs_to_dialog(d)

file_train = open(f"{data_path}train.txt","w")
print("Training Set:", file_train.write(data_train))
#endregion 
    
#region Validation
data_valid = ""
for d in dialogs_valid:
  data_valid += transfer_list_of_pairs_to_dialog(d)

file_valid = open(f"{data_path}valid.txt","w")
print("Validation Set:", file_valid.write(data_valid))
#endregion

#region Test
data_test = ""
for d in dialogs_test:
  data_test += transfer_list_of_pairs_to_dialog(d)

file_test = open(f"{data_path}test.txt","w")
print("Test Set:", file_test.write(data_test))
#endregion 

# print(len(data_valid))

Training Set: 15776427
Validation Set: 257114
Test Set: 262425


## ED

In [ ]:
googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_empathetic_dialogues/"
data_path = "/content/dataset_french_ed/"
!rm -R $data_path
!mkdir $data_path
!cp -rv $googledrive_data_path* $data_path


## BST

In [ ]:
googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/"
data_path = "/content/dataset_french_bst/"
!rm -R $data_path
!mkdir $data_path
!cp -rv $googledrive_data_path* $data_path



# 2. Creating new Tasks

In [ ]:
#region XPersona
task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

!rm -R $task_path'french_xpersona/'
!mkdir $task_path'french_xpersona'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/* $task_path'french_xpersona/'
#endregion


#region ED
task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

!rm -R $task_path'french_empathetic_dialogues/'
!mkdir $task_path'french_empathetic_dialogues'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_empathetic_dialogues/* $task_path'french_empathetic_dialogues/'
#endregion


#region BST
task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

!rm -R $task_path'french_blended_skill_talk/'
!mkdir $task_path'french_blended_skill_talk'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_blended_skill_talk/* $task_path'french_blended_skill_talk/'
#endregion

# 3. Display Model

In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='empathetic_dialogues',
    model_file= f'{from_scratch_model_path}/model',
    num_examples=8,
    # skip_generation=False,
)

# 3. Finetuning

In [ ]:
finetuned_model_path = f'{mydrive_path}finetuned-multitask-90m'
init_model = 'zoo:blender/blender_90M/model'
dict_file  = 'zoo:blender/blender_90M/model.dict'

In [ ]:
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    task='fromfile:parlaiformat', 
    fromfile_datapath='fr_finetuned',
    fromfile_datatype_extension=True,

    model='transformer/generator',
    model_file= f'{finetuned_model_path}/model',
    
    # initialize with a pretrained model
    init_model= init_model,
    
    # arguments we get from the pretrained model.
    # Unfortunately, these must be looked up separately for each model.
    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    dict_file=dict_file,
    
    # some training arguments, specific to this fine-tuning
    # use a small learning rate with ADAM optimizer
    lr=1e-5, optimizer='adam',
    warmup_updates=100,
    validation_metric='ppl',
    validation_every_n_epochs=0.25,
    num_epochs = 3,
    verbose = True,
    
    # depend on your gpu. If you have a V100, this is good
    batchsize=8, fp16=True, fp16_impl='mem_efficient',
    
    # speeds up validation
    skip_generation=True,
    
    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',

    # --beam-min-length 20 --beam-block-ngram 3 --beam-context-block-ngram 3 --beam-size 10 --inference beam
    beam_min_length= 20,
    beam_block_ngram= 3,
    beam_context_block_ngram= 3,
    beam_size= 10,

    inference= "beam"
)